# Leonardo's Programmable Loom: The First Computer?

> *"This is the way to weave..."*  
> — Leonardo da Vinci

## Introduction
Long before the Jacquard loom or the Babbage engine, Leonardo sketched a loom controlled by a **cam barrel**. This rotating drum had pegs that acted as instructions, lifting specific warp threads to create complex patterns automatically.

In this notebook, we explore:
1.  **Mechanical Programming**: How pegs and cams store information.
2.  **Weaving Logic**: Generating patterns from binary (lift/lower) instructions.
3.  **Fabric Simulation**: Visualizing the resulting textile.

---

In [ ]:
# Install the davinci-codex library
!pip install -q git+https://github.com/Shannon-Labs/davinci-codex.git

import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widgets
from ipywidgets import interact

# Set "Brutalist Academic" plotting style
plt.rcParams.update({
    'font.family': 'serif',
    'font.serif': ['Times New Roman', 'DejaVu Serif'],
    'axes.grid': True,
    'grid.alpha': 0.3,
    'axes.facecolor': 'white',
    'figure.facecolor': 'white',
    'text.color': 'black',
    'axes.labelcolor': 'black',
    'xtick.color': 'black',
    'ytick.color': 'black'
})


## The Physics Model

### 1. Binary Logic
Each peg on the cam barrel corresponds to a binary state for a group of warp threads:
- **1 (Lift)**: Warp thread is raised, weft goes *under*.
- **0 (Lower)**: Warp thread is lowered, weft goes *over*.

### 2. Pattern Generation
A pattern is a matrix $P_{i,j}$ where $i$ is the weft row (time step) and $j$ is the warp column (thread index).
The cam barrel rotates, advancing $i$ with each pick.

In [ ]:
def generate_pattern(pattern_type, size=32):
    matrix = np.zeros((size, size))
    
    for i in range(size):
        for j in range(size):
            if pattern_type == 'Plain':
                # Checkerboard
                matrix[i, j] = (i + j) % 2
            elif pattern_type == 'Twill':
                # Diagonal
                matrix[i, j] = (i + j) % 4 < 2
            elif pattern_type == 'Satin':
                # Sparse intersections
                matrix[i, j] = (i * 3 + j) % 5 == 0
            elif pattern_type == 'Leonardo':
                # Complex geometric
                x, y = j/size - 0.5, i/size - 0.5
                r = np.sqrt(x**2 + y**2)
                matrix[i, j] = (np.sin(20*r) > 0) ^ ((i//4)%2 == (j//4)%2)
                
    return matrix

def plot_loom(pattern_type, zoom):
    size = 64
    matrix = generate_pattern(pattern_type, size)
    
    # Zoom
    view_size = int(size / zoom)
    view_matrix = matrix[:view_size, :view_size]
    
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 6))
    
    # Plot 1: Cam Barrel Instruction (The "Code")
    # Visualize as a punch card / peg drum strip
    ax1.imshow(view_matrix, cmap='binary', interpolation='nearest', aspect='auto')
    ax1.set_title('Cam Barrel Instructions (1=Peg, 0=No Peg)')
    ax1.set_xlabel('Warp Thread Index')
    ax1.set_ylabel('Time Step (Weft Row)')
    
    # Plot 2: Fabric Simulation
    # Visualize as woven threads
    # Weft (horizontal) is typically one color, Warp (vertical) another
    
    # Create RGB image
    img = np.zeros((view_size, view_size, 3))
    
    warp_color = np.array([0.8, 0.2, 0.2]) # Red Silk
    weft_color = np.array([0.9, 0.9, 0.8]) # Natural Linen
    
    for i in range(view_size):
        for j in range(view_size):
            if view_matrix[i, j] == 1:
                # Warp on top
                img[i, j] = warp_color
            else:
                # Weft on top
                img[i, j] = weft_color
                
            # Add some shading for 3D effect
            # Simple edge darkening
            # (Not implemented for simplicity in this view)
                
    ax2.imshow(img, interpolation='nearest')
    ax2.set_title('Woven Fabric Simulation')
    ax2.axis('off')
    
    plt.tight_layout()
    plt.show()
    
    complexity = np.mean(view_matrix)
    print(f"Pattern Density: {complexity*100:.1f}% Warp Lifts")
    print(f"Instructions Required: {view_size*view_size} bits")

interact(plot_loom, 
         pattern_type=widgets.Dropdown(options=['Plain', 'Twill', 'Satin', 'Leonardo'], value='Leonardo', description='Pattern'),
         zoom=widgets.FloatSlider(min=1.0, max=4.0, step=0.5, value=1.0, description='Zoom'));

## Conclusion

Leonardo's loom demonstrates the fundamental concept of **programmable automation**. By separating the *mechanism* (the loom) from the *information* (the cam barrel), he anticipated the separation of hardware and software that defines modern computing.